In [6]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
from keras.models import load_model


In [ ]:
model = load_model("cifar4_CNN_Epoch_100_Daug_True.h5")
model.summary()

In [ ]:
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

score = model.evaluate(x_test,y_test)
score

In [ ]:
import foolbox
import numpy as np
import UtilHelper as utilh

foolmodel = foolbox.models.KerasModel(model, bounds=(0.0,1.0))

In [ ]:
criterion = foolbox.criteria.Misclassification()
attack = foolbox.attacks.FGSM(foolmodel, criterion)

ind = 654
image = x_test[ind]
label = np.argmax(y_test[ind])
test = model.predict(np.reshape(image,(1,32,32,3)))

image.shape
utilh.plotImage(image*255)

adversarial = attack(image, label,unpack=False)
adversarial_image = adversarial.image

In [ ]:
adversarial_predict = model.predict(np.reshape(adversarial_image,(1,32,32,3)))
print("adv predict: " + str(np.argmax(adversarial_predict)))
test_predict = model.predict(np.reshape(image,(1,32,32,3)))
print("test predict: "+ str(np.argmax(test_predict)))
label
print("orj label: " + str(label))

In [ ]:
utilh.plotAllImages(image*255, adversarial_image*255, np.argmax(adversarial_predict), label)

In [ ]:
from IPython.display import clear_output
x_advlist = []
y_advlist = []
for i in range(0,len(x_train)):
    img = x_train[i]
    label = np.argmax(y_train[i])
    adversarial = attack(img, label, unpack=False)
    # in some circumstances, attacks will be failed, control if success
    if(adversarial is not None) and (adversarial.image is not None):
        if i % (len(x_train)/1000) == 0:
            clear_output(wait=True)
            print(str(i / (len(x_train)/100)) +"%")
        x_advlist.append(adversarial.image)
        y_advlist.append(y_train[i])

In [ ]:
np.array(x_advlist)

In [ ]:
model.evaluate(np.asarray(x_advlist),np.asarray(y_advlist),verbose=1)

In [ ]:
np.save("adversarial_x_train",np.asarray(x_advlist))
np.save("adversarial_y_train",np.asarray(y_advlist))